In [63]:
%load_ext autoreload
%autoreload 2

In [43]:
import numpy as np
import quant
from functools import reduce
from random import randint

In [31]:
def create_qft_matrix(n):
    """
    Generates Quantum Fourier Transformation matrix 
    
    Parameters
    ----------
    n: int
        Number of qubits to be transformed
    
    Return
    ------
    
    qft: array_like
        QFT matrix
    """
    N = 2 ** n
    w = croot(N, 1)
    norma = 1 / np.sqrt(N)
    res = np.empty([N, N], dtype=np.complex_)
    
    for x in range(N):
        
        temp = np.zeros([N, 1], dtype=np.complex_)
        
        for k in range(N):
            w_xk = w ** (x * k % N)
            vector_k = format(k, '0' + str(n) + 'b')
            column = quant.string_to_state(vector_k)
            temp += w_xk * column
            
        res[:, x] = temp[:, 0]
    
    return norma * res

In [32]:
def croot(n, k):
    if n<=0:
        return None
    return np.exp((2 * np.pi * 1j * k) / n)
    

## Step 1 

In [33]:
N = 15

In [34]:
input_q = 4 
output_q = 4

In [35]:
x_0n = reduce(np.kron, np.repeat(quant.ZERO[np.newaxis, :], input_q, axis=0))
y_0n = reduce(np.kron, np.repeat(quant.ZERO[np.newaxis, :], output_q, axis=0))
input_vector = np.kron(x_0n, y_0n)

## Step 2
Hadamard stage

In [36]:
H = quant.hadamard_n(input_q)
I = reduce(np.kron, np.repeat(np.eye(2)[np.newaxis, :], output_q, axis=0))
HI = np.kron(H, I)

In [37]:
input_vector_s2 = np.dot(HI, input_vector)

## Step 3
Function 
$$ a^x mod N $$


In [38]:
def create_number_theory_matrix(n, func):
    N = 2 ** n
    res = np.empty([N, N])
    mid = n // 2 
    
    for num in range(N):
        state = format(num, '0' + str(n) + 'b')
        X = state[:mid]
        Y_ = int(state[mid:], 2) ^ func(int(X, 2))
        Y = format(Y_, '0' + str(mid) + 'b')
        column = quant.string_to_state(X + Y)
        
        res[:, num] = column[:, 0]
    
    return res  

In [39]:
def get_coprime(N):
    while True:
        a = randint(2, N-1)
        if gcd(a, N) == 1:
            return a

In [40]:
def gcd(a, b):
    while b != 0:
        tA = a % b
        a = b
        b = tA

    return a

In [41]:
def U_func(a, N):
    return lambda x : (a ** x) % N

In [50]:
a = get_coprime(N)
print(a)

7


In [51]:
U_f = create_number_theory_matrix(input_q + output_q, U_func(a, N))

In [52]:
input_vector_s3 = np.dot(U_f, input_vector_s2)

## Step 4

In [53]:
QFT = create_qft_matrix(input_q + output_q)

In [54]:
input_vector_s4 = np.dot(QFT, input_vector_s3)

## Step 5
Measuring

In [68]:
states = 2 ** input_q
t = 0

maxProb = 0
meas = 0

for num in range(states):
    basis_str = format(num, '0' + str(input_q) + 'b')
    
    basis = quant.string_to_state(basis_str)
    prob = quant.measure(input_vector_s4, basis)
    t += prob
    print('|' + basis_str + '>', prob)
    
    if prob > maxProb:
        maxProb = prob
        meas = num
print(t)

|0000> (0.1064212711924642-1.4207642135853447e-19j)
|0001> (0.04463590331320625-9.796316366645934e-19j)
|0010> (0.04501521851717494+3.0697800232125416e-19j)
|0011> (0.08851544143941953+4.5453042605161975e-19j)
|0100> (0.05705830108336861-2.9631525379425685e-20j)
|0101> (0.05569273951595075+2.6626393125647376e-19j)
|0110> (0.04229091088300389+4.540965756170749e-19j)
|0111> (0.08380771405541383-1.259522641129069e-18j)
|1000> (0.09361787729441373+3.683474545153291e-19j)
|1001> (0.03476897591598383-3.2219016622755303e-19j)
|1010> (0.024930635777089715+2.3820771658267202e-20j)
|1011> (0.09334467655025061+8.060109086413044e-19j)
|1100> (0.08707972999422156+8.609133927012375e-19j)
|1101> (0.05197520169039383-7.755644862043816e-19j)
|1110> (0.04358605525826584-4.6514613032161055e-20j)
|1111> (0.047259347519384744-8.749145583926526e-20j)
(1.0000000000000058-1.0166148307242264e-19j)


In [74]:
r = cf(2, input_q+output_q, N)

In [77]:
for i in range(16):
    print(cf(i, input_q+output_q, N))

0
8
4
8
2
8
4
8
0
8
4
8
2
8
4
8


In [66]:
def cf(y, Q, N):
    fractions = extendedGCD(y, Q)
    depth = 2

    def partial(fractions, depth):
        c = 0
        r = 1

        for i in reversed(range(depth)):
            tR = fractions[i] * r + c
            c = r
            r = tR

        return c

    r = 0
    for d in range(depth, len(fractions) + 1):
        tR = partial(fractions, d)
        if tR == r or tR >= N:
            return r

        r = tR

    return r

In [67]:
# Extended Euclidean
def extendedGCD(a, b):
    fractions = []
    while b != 0:
        fractions.append(a // b)
        tA = a % b
        a = b
        b = tA

    return fractions

In [11]:
res

array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 2., 3., 4., 5., 6., 7.],
       [0., 2., 4., 6., 0., 2., 4., 6.],
       [0., 3., 6., 1., 4., 7., 2., 5.],
       [0., 4., 0., 4., 0., 4., 0., 4.],
       [0., 5., 2., 7., 4., 1., 6., 3.],
       [0., 6., 4., 2., 0., 6., 4., 2.],
       [0., 7., 6., 5., 4., 3., 2., 1.]])

In [21]:
a = create_qft_matrix(3)

In [30]:
a[1, 6]

(-6.661338147750939e-16-1j)